In [ ]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [ ]:
## 安裝套件
!pip install pypdf pyngrok  openai gdown chainlit langchain chromadb pdfplumber tiktoken  cohere -q

In [ ]:
# INPUT OPENAPI KEY
import os
OPENAI_API_KEY='sk-xxxxxxxxxxxxxxxxxxxxxx'
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from typing import List

from tempfile import NamedTemporaryFile

import chainlit as cl
from chainlit.types import AskFileResponse
import chromadb
from chromadb.config import Settings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PDFPlumberLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import Document
from langchain.schema.embeddings import Embeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.vectorstores.base import VectorStore
from c01_prompts import EXAMPLE_PROMPT, PROMPT, WELCOME_MESSAGE

In [ ]:
# read pdf
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_file='./data/1Q23-EPR-with-Tables-FINAL.pdf'
loader= PyPDFLoader(pdf_file)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=100
)
docs = text_splitter.split_documents(documents)

for i, doc in enumerate(docs):
    doc.metadata["source"] = f"source_{i}"

In [ ]:
# embedding
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
# DB
from langchain.vectorstores import Chroma

persist_directory="./chromadb/"
search_engine = Chroma.from_documents(
    documents=docs, # text data that you want to embed and store
    embedding=embeddings, # used to convert the documents into embeddings
    #persist_directory=persist_directory, # this tells Chroma where to store its data
    collection_name="mydb" #  gives a name to the collection of embeddings, which will be helpful for retrieving specific groups of embeddings later.
)

In [ ]:
# llm model
from langchain.chains import RetrievalQAWithSourcesChain

llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k-0613", temperature=0, streaming=True
)

In [ ]:
# RAG Chain
from langchain.chains import RetrievalQAWithSourcesChain
from c01_prompts import EXAMPLE_PROMPT, PROMPT, WELCOME_MESSAGE
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=search_engine.as_retriever(max_tokens_limit=512),
    chain_type_kwargs={"prompt": PROMPT, "document_prompt": EXAMPLE_PROMPT},
    return_source_documents=True
)

In [ ]:
## DO QA Search
query = "What's the results for the reporter quarter? Please describe in the following order using bullet points - revenue, gross margin, opex, op margin, net income, and EPS. INclude both gaap and non-gaap numbers. Please also include quarter over quarter changes."
llm_response = chain(query)
print(llm_response['question'].strip())
print('\n\n\n')
print(llm_response['answer'].strip())
print('\n\n\n')
print(llm_response['sources'].strip())
print('\n\n\n')
print(llm_response['source_documents'])

## DEMO

In [ ]:
!chainlit run c01_app.py -w

In [ ]:
!ngrok config add-authtoken 1wbrOL2j8XoIkeqLu26tSNIOb6D_4cq3RGaNcGTt7v6G4nQX5

In [ ]:
from pyngrok import ngrok
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

In [ ]:
ngrok.kill()

In [ ]:
!ps -ef |grep chainlit | awk '{print $2}' | xargs kill -9
!ps -ef |grep ngrok | awk '{print $2}' | xargs kill -9